In [1]:
import tkinter as tk
from tkinter import simpledialog, messagebox
import pandas as pd
import os
import subprocess
import sys
from datetime import datetime


def get_user_inputs():
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    valid_event_types = {'ESPP', '401k'}
    valid_months = {'April', 'October'}

    while True:
        data_pull_type = simpledialog.askstring("Data Pull Type", "Enter data pull type (ESPP or 401k):")
        if data_pull_type not in valid_event_types:
            messagebox.showerror("Invalid Input", "Please enter a valid data pull type (ESPP or 401k).")
            continue

        month_option = simpledialog.askstring("Month Option", "Choose month (April or October):")
        if month_option not in valid_months:
            messagebox.showerror("Invalid Input", "Please enter a valid month (April or October).")
            continue

        year = simpledialog.askinteger("Year", "Enter the year:")
        current_year = datetime.now().year
        if year and 1900 <= year <= current_year + 1:
            break
        else:
            messagebox.showerror("Invalid Input", "Please enter a valid year.")
    
    root.destroy()
    return data_pull_type, month_option, year

def calculate_date_range(month_option, year):
    if month_option == "April":
        start_date = f"{year - 1}-10-16"
        end_date = f"{year}-04-15"
    else:  # October
        start_date = f"{year}-04-16"
        end_date = f"{year}-10-15"
    return start_date, end_date

def filter_attendees(data_pull_type, start_date, end_date, attendee_db):
    filtered_attendees = attendee_db[
        (attendee_db['Event Type'] == data_pull_type) &
        (attendee_db['Event Date'] >= start_date) &
        (attendee_db['Event Date'] <= end_date)
    ][['EID Extract', 'Event Type', 'Event Date']]
    # Remove duplicate entries based on 'EID Extract'
    unique_attendees = filtered_attendees.drop_duplicates(subset=['EID Extract'])
    return unique_attendees

def create_unique_filename(base_filename):
    version = 1
    filename, file_extension = os.path.splitext(base_filename)
    new_filename = base_filename

    while os.path.exists(new_filename):
        new_filename = f"{filename}_{version}{file_extension}"
        version += 1

    return new_filename

def save_and_open_csv(data, data_pull_type, month_option, year):
    base_filename = f"DataPull-{data_pull_type}.{month_option}.{year}.csv"
    unique_filename = create_unique_filename(base_filename)

    data.to_csv(unique_filename, index=False)
    try:
        os.startfile(unique_filename)  # For Windows
    except AttributeError:
        if sys.platform == 'darwin':  # For MacOS
            subprocess.call(['open', unique_filename])
        else:  # For Linux
            subprocess.call(['xdg-open', unique_filename])

def main():
    data_pull_type, month_option, year = get_user_inputs()
    start_date, end_date = calculate_date_range(month_option, year)
    attendee_db_path = "Attendee-Database.csv"

    attendee_db_path = "Attendee-Database.csv"
    if not os.path.exists(attendee_db_path):
        messagebox.showerror("File Not Found", "Attendee database file not found. Please check the file path.")
        return

    attendee_db = pd.read_csv(attendee_db_path)
    attendees = filter_attendees(data_pull_type, start_date, end_date, attendee_db)
    save_and_open_csv(attendees, data_pull_type, month_option, year)

if __name__ == "__main__":
    main()


not working.. 

# Import necessary libraries
import pandas as pd
from openpyxl import load_workbook
import os
from tkinter import messagebox

# Function to append data to an Excel workbook
def append_to_excel(csv_file, excel_file, sheet_name, start_row=5):
    df = pd.read_csv(csv_file)
    workbook = load_workbook(excel_file)
    sheet = workbook[sheet_name]
    for i, row in enumerate(df.itertuples(index=False), start=start_row):
        sheet.cell(row=i, column=1, value=row[0])  # Assumes 'EID Extract' is the first column
    workbook.save(excel_file)

# Main function for the appending process
def main():
    # Get user inputs for filename generation
    data_pull_type, month_option, year = get_user_inputs()

    # Generate the filename based on user inputs
    base_filename = f"DataPull-{data_pull_type}.{month_option}.{year}.csv"
    csv_filename = os.path.join('path/to/csv', base_filename)  # Update with the correct folder path

    # Check if the CSV file exists
    if not os.path.exists(csv_filename):
        messagebox.showerror("File Not Found", f"CSV file {csv_filename} not found. Please run the data pull script first.")
        return

    # Append data to the Excel workbook
    excel_filename = 'YourExcelWorkbook.xlsx'  # Replace with your actual Excel workbook filename
    append_to_excel(csv_filename, excel_filename, 'Template-New Participants')

# Execute the main function
if __name__ == "__main__":
    main()

pip install openpyxl is required for writing to excel files